In [9]:
import pandas as pb 
import requests
import pyspark
from pyspark import SparkConf                                                                                                                 
from pyspark.context import SparkContext                                                                                                      
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.session import SparkSession

In [10]:
#!PIP install pyspark
#!PIP install Spark


In [11]:

def get_api_key(api_key_id = "Realtor"):
  """
  Get the api key for website accessing.

  Table of key type and key value for privacy.

  Parameters
  ----------
  @api_key_id [string]: Key value in dataframe

  Returns
  -------
  [string]: API Key

  """
  # load api keys file
df_api_keys = spark.read.format('csv').options(header='true', inferSchema='true').load('FileStore/tables/api_keys.csv').toPandas()

#df_api_keys = spark.read.csv('FileStore/tables/api_keys.csv').toPandas()
  
    # return api key if in dataset
try:
    # get api key from id
    api_key = df_api_keys.loc[df_api_keys['Id'] == api_key_id]['Key'].iloc[0] # get key by id
    # return api key
    return api_key

except IndexError:
    # get api key id list
    api_key_id_list = df_api_keys['Id'].unique().tolist()
    # print error message
    print('Cannot map key. Api key id must be one of the following options {0}'.format(api_key_id_list))

NameError: name 'spark' is not defined

In [ ]:

def api_property_list_for_sale(api_key, city, state, prop_type, limit=200):
  # url for api
  url = "https://realtor-com-real-estate.p.rapidapi.com/for-sale"

  # enter parameters
  querystring = {
    "sort":"relevance",
    "city":city,
    "offset":"0",
    "limit":limit,
    "state_code":state,
    "prop_type":prop_type
  }

  # header
  headers = {
    'x-rapidapi-host': "realtor.p.rapidapi.com",
    'x-rapidapi-key': api_key
  }

  # response
  response = requests.request("GET", url, headers=headers, params=querystring)
  return response.json() # json format

In [ ]:
def process_list_for_sale_response(response_json):
    """
    Process the list for sale API response.

    Convert each listing to a dataframe, append to a list, and concatenate to one dataframe.

    Parameters
    ----------
    @response_json [dictionary]: API response for list for sale

    Returns
    -------
    [dataframe] Dataframe of all list for sale responses

    """

    # empty dataframe
    dataframe_list = []

    # iterate through each for sale listing
    for l in response_json['properties']:

        # convert each listing to dataframe
        _temp_df = pd.DataFrame.from_dict(l, orient='index').T

        # append to dataframe list for all listings
        dataframe_list.append(_temp_df)

    # concatenate all dataframes, for missing col values enter null value
    return pd.concat(dataframe_list, axis=0, ignore_index=True, sort=False)


In [ ]:

# api key to accessd ata
realtor_api_key = get_api_key(api_key_id = "Realtor")
city = "Nashville"
state = "TN"
prop_type = "single_family"